In [1]:
## PART 3
import mysql.connector
from mysql.connector.constants import ClientFlag

config = {
    'user': 'mehdi',
    'password': 'Sz2295780211',
    'host': 'localhost',
    'database': 'data',
    'auth_plugin' : 'mysql_native_password'
}

cnx = mysql.connector.connect(**config)
cnx.autocommit = True
cursor = cnx.cursor()

In [2]:
def markdown_table(res, titles):
    if len(titles) != len(res[0]):
        raise Exception("arg arrays don't match")

    table = ' | '.join(titles)
    table += '\n'
    table += ' | '.join([":---:"] * len(titles))
    table += '\n'

    for row in res:
        table += ' | '.join([str(elem) for elem in row])
        table += '\n'

    return table

In [21]:
##QUERY2
cursor.execute("""DESCRIBE 
SELECT vt.description , COUNT(*)
FROM(
    SELECT p.id 
    FROM Collisions c, Factors f, RoadCondition r, Parties p
    WHERE c.case_id = f.case_id AND( 
        r.id = f.road_condition_1 OR 
        r.id = f.road_condition_2) AND
        r.description = 'holes' AND
        c.case_id = p.case_id ) AS parties_holes , Vehicles v , StatewideVehiculeType vt
WHERE parties_holes.id = v.party_id AND
      vt.id = v.statewide_vehicle_type
GROUP BY v.statewide_vehicle_type
ORDER BY COUNT(*) DESC
LIMIT 5
      """)

In [22]:
res = cursor.fetchall()

In [23]:
print(res)

[(1, 'SIMPLE', 'r', None, 'const', 'PRIMARY,description', 'description', '82', 'const', 1, 100.0, 'Using index; Using temporary; Using filesort'), (1, 'SIMPLE', 'f', None, 'index_merge', 'road_condition_1,road_condition_2,case_id', 'road_condition_1,road_condition_2', '5,5', None, 17918, 100.0, 'Using union(road_condition_1,road_condition_2); Using where'), (1, 'SIMPLE', 'c', None, 'eq_ref', 'PRIMARY', 'PRIMARY', '78', 'data.f.case_id', 1, 100.0, 'Using index'), (1, 'SIMPLE', 'p', None, 'ref', 'PRIMARY,case_id', 'case_id', '78', 'data.f.case_id', 1, 100.0, 'Using index'), (1, 'SIMPLE', 'v', None, 'ref', 'statewide_vehicle_type,party_id', 'party_id', '4', 'data.p.id', 1, 100.0, 'Using where'), (1, 'SIMPLE', 'vt', None, 'eq_ref', 'PRIMARY', 'PRIMARY', '4', 'data.v.statewide_vehicle_type', 1, 100.0, None)]


In [10]:
###QUERY 3
cursor.execute("""SELECT
  make.description,
  COUNT(*)
FROM
  Vehicles v,
  Victims vict,
  Parties p,
  VehiculeMake make,
  VictimDegreeOfInjury injury
WHERE
  v.party_id = p.id
  AND vict.case_id = p.case_id
  AND v.vehicle_make = make.id
  AND vict.victim_degree_of_injury = injury.id
  AND (
    injury.description = 'Killed'
    OR injury.description = 'Severe Injury'
  )
GROUP BY
  v.vehicle_make
ORDER BY
  COUNT(*) DESC
LIMIT 10
""")


In [77]:
res = cursor.fetchall()

In [78]:
 for x in res :
     print(x)

('FORD', 6581)
('CHEVROLET', 4901)
('TOYOTA', 4333)
('HONDA', 3889)
('DODGE', 1990)
('NISSAN', 1626)
('GMC', 1083)
('NOT STATED', 927)
('MISCELLANEOUS', 822)
('FREIGHTLINER', 810)


In [4]:
###QUERY 4
cursor.execute("""SET @v1:=(SELECT COUNT(*)
                            FROM Collisions c, Victims v ,VictimDegreeOfInjury vd
                            WHERE c.case_id = v.case_id                     AND 
                                    v.victim_degree_of_injury = vd.id         AND
                                    vd.description = 'No Injury' )          
                            """)
cursor.execute("""

    CREATE VIEW Position_Safety AS (
        SELECT v.victim_seating_position as seating, COUNT(*) as pourcentage
        FROM Collisions c, Victims v ,VictimDegreeOfInjury vd
        WHERE c.case_id = v.case_id                     AND 
            v.victim_degree_of_injury = vd.id         AND
            vd.description = 'No Injury'              AND
            v.victim_seating_position IS NOT NULL
            
        GROUP BY v.victim_seating_position  
        ORDER BY COUNT(*) DESC ) 
""")


In [3]:
cursor.execute("""DROP VIEW Position_Safety""")

In [10]:
cursor.execute("""
SELECT victim_seating_position, pourcentage/@v1 
FROM Position_Safety  
WHERE pourcentage/@v1 = (SELECT MAX(pourcentage/@v1)FROM Position_Safety) OR 
      pourcentage/@v1 = (SELECT MIN(pourcentage/@v1)FROM Position_Safety) 
""") 

In [5]:
cursor.execute("""
SELECT *
FROM Position_Safety
""")
res = cursor.fetchall()


In [7]:
for x in res:
    print (x)

(3, 963439)
(6, 358747)
(4, 292640)
(5, 136898)
(9, 70469)
(2, 45875)
(0, 45104)
(7, 42601)
(8, 33752)
(1, 11903)


In [46]:
print(markdown_table(res,['Seating','Safety %']))

Seating | Safety %
:---: | :---:
3 | 0.4812
1 | 0.0059



In [43]:
#QUERY5
cursor.execute("""
SELECT COUNT(*)
FROM (SELECT statewide_vehicle_type  ,COUNT(Parties.case_id) as collisions ,COUNT(DISTINCT county_city_location) as city
      FROM (Vehicles   INNER JOIN Parties ON Parties.id = Vehicles.party_id 
                       INNER JOIN Locations ON Locations.case_id = Parties.id) 
GROUP BY statewide_vehicle_type 
) A
WHERE A.collisions > 10 AND
      A.city > (
SELECT COUNT(DISTINCT county_city_location)
FROM Locations)/2
                """)







In [46]:
res = cursor.fetchall()

In [47]:
print(res)

[(65,)]


In [10]:
###QUERY6
cursor.execute("""
    SELECT l.county_city_location , l.population , l.case_id
    FROM Locations l
    WHERE l.population <= @population
""")

In [28]:
print(res)

, (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4,), (4

In [20]:
print(res)

[(9,)]


In [8]:
###QUERY7
cursor.execute("""
SELECT
  old_people_collisions.id,
  old_people_collisions.max_age
FROM(
    SELECT
      v.case_id AS id,
      MAX(v.victim_age) AS max_age
    FROM
      Victims v
    GROUP BY
      v.case_id
    HAVING
      MIN(v.victim_age) > 99
  ) AS old_people_collisions,
  Collisions c,
  TypeOfCollision tc
WHERE(
    c.case_id = old_people_collisions.id
    AND c.type_of_collision = tc.id
    AND tc.description = 'pedestrian'
  )
""")

In [9]:
res = cursor.fetchall()

In [10]:
print(markdown_table(res,['case_id','age']))

case_id | age
:---: | :---:
0036446 | 110
0069198 | 101
0415838 | 100
0439197 | 102
0445265 | 101
0486529 | 100
0566220 | 102
0621752 | 100
0644226 | 103
0784061 | 102
0817210 | 102
0820619 | 101
0828116 | 102
0851026 | 106
0868472 | 103
0885420 | 100
1209166 | 101
1213340 | 121
1347636 | 101
1373664 | 101
1548445 | 102
1847678 | 104
2290129 | 100
2427260 | 100
2472739 | 103
2531557 | 103
3388544 | 105
3485436 | 101



In [27]:
###QUERY8
cursor.execute("""
SELECT
  makes.description,
  v.vehicle_year,
  type.description,
  COUNT(*) AS num_of_collisions
FROM
  Vehicles v,
  Parties p,
  Collisions c,
  VehiculeMake makes,
  StatewideVehiculeType type
WHERE
  v.party_id = p.id AND v.vehicle_make != 'None'
  AND p.case_id = c.case_id AND
  v.vehicle_make = makes.id AND
  v.statewide_vehicle_type = type.id
GROUP BY v.vehicle_make,
  v.vehicle_year,
  v.statewide_vehicle_type
HAVING(COUNT(*) > 9)
ORDER BY
  num_of_collisions DESC
""")

In [28]:
res = cursor.fetchall()

In [33]:
for x in res:
    print(x)

In [3]:
###QUERY9
cursor.execute("""
SELECT
  l.county_city_location,
  COUNT(*)
FROM
  Collisions c,
  Locations l
WHERE
  (c.case_id = l.case_id)
GROUP BY
  (l.county_city_location)
ORDER BY
  COUNT(*) DESC
LIMIT
  10
""")

In [4]:
res = cursor.fetchall()

In [6]:
print(markdown_table(res,['city','count']))

city | count
:---: | :---:
1942 | 399582
1900 | 118446
3400 | 80191
3711 | 76867
109 | 72995
3300 | 61453
3404 | 58068
4313 | 57852
1941 | 53565
3801 | 48450



In [3]:
###QUERY 10
cursor.execute("""
SELECT
  CASE WHEN l.description = 'daylight'
  OR (
    f.lighting = NULL
    AND c.collision_time BETWEEN '00:08:00'
    AND '17:59:00'
  ) THEN 'day' WHEN (
    f.lighting = NULL
    AND c.collision_time BETWEEN '00:06:00'
    AND '07:59:00'
  )
  OR (
    l.description = 'dusk or dawn'
    AND (
      c.collision_time BETWEEN '00:06:00'
      AND '07:59:00'
    )
  ) THEN 'dawn' WHEN (
    f.lighting = NULL
    AND c.collision_time BETWEEN '00:18:00'
    AND '19:59:00'
  )
  OR (
    l.description = 'dusk or dawn'
    AND (
      c.collision_time BETWEEN '00:18:00'
      AND '19:59:00'
    )
  ) THEN 'dusk' ELSE 'night' END AS time_of_day,
  count(1) as num
FROM
  Factors f,
  Lighting l,
  Collisions c
WHERE
  f.case_id = c.case_id
  AND (
    f.lighting = l.id
    OR f.lighting = NULL
  )
GROUP BY
  time_of_day

""")

In [4]:
res = cursor.fetchall()

In [5]:
print(markdown_table(res,['day_time','count']))

day_time | count
:---: | :---:
night | 1039682
day | 2496938
dusk | 74906
dawn | 40756

